<a href="https://colab.research.google.com/github/MarinaWolters/Coding-Tracker/blob/master/W12_Streamparse_Low_Level_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low-Level Streaming


This notebook focuses on *StreamParse*, which is a Python library for using Apache Storm.

Storm and its relatives process tuples one-at-a-time.

We'll see StreamParse in action, then look at some functions written **for** StreamParse but executed outside, to get better ideas of how they work.

## Installation of Storm, StreamParse and Their Support Libraries

In [1]:
!wget https://raw.githubusercontent.com/technomancy/leiningen/stable/bin/lein

--2023-04-21 22:13:40--  https://raw.githubusercontent.com/technomancy/leiningen/stable/bin/lein
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13958 (14K) [text/plain]
Saving to: ‘lein’

lein                100%[===================>]  13.63K  --.-KB/s    in 0s      

2023-04-21 22:13:41 (120 MB/s) - ‘lein’ saved [13958/13958]



In [11]:
!wget https://dlcdn.apache.org/storm/apache-storm-1.2.4/apache-storm-1.2.4.tar.gz

--2023-04-21 22:18:37--  https://dlcdn.apache.org/storm/apache-storm-1.2.4/apache-storm-1.2.4.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169097858 (161M) [application/x-gzip]
Saving to: ‘apache-storm-1.2.4.tar.gz’

apache-storm-1.2.4. 100%[===================>] 161.26M   197MB/s    in 0.8s    

2023-04-21 22:18:46 (197 MB/s) - ‘apache-storm-1.2.4.tar.gz’ saved [169097858/169097858]



In [12]:
!tar zxvf apache-storm-1.2.4.tar.gz

apache-storm-1.2.4/bin/storm-config.cmd
apache-storm-1.2.4/bin/storm.ps1
apache-storm-1.2.4/bin/storm-kafka-monitor
apache-storm-1.2.4/bin/storm.cmd
apache-storm-1.2.4/bin/storm.py
apache-storm-1.2.4/bin/storm
apache-storm-1.2.4/bin/flight.bash
apache-storm-1.2.4/public/index.html
apache-storm-1.2.4/public/topology.html
apache-storm-1.2.4/public/js/script.js
apache-storm-1.2.4/public/deep_search_result.html
apache-storm-1.2.4/public/component.html
apache-storm-1.2.4/public/logviewer_search.html
apache-storm-1.2.4/public/flux.html
apache-storm-1.2.4/public/supervisor.html
apache-storm-1.2.4/public/search_result.html
apache-storm-1.2.4/public/css/
apache-storm-1.2.4/public/images/
apache-storm-1.2.4/public/js/
apache-storm-1.2.4/public/templates/
apache-storm-1.2.4/public/favicon.ico
apache-storm-1.2.4/public/css/bootstrap-3.3.1.min.css
apache-storm-1.2.4/public/css/jsonFormatter.min.css
apache-storm-1.2.4/public/css/style.css
apache-storm-1.2.4/public/css/dataTables.bootstrap.css
apache

In [14]:
!chmod a+x ./lein
!chmod a+x apache-storm-1.2.4/bin/storm

In [15]:
!bash lein version

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12.1M  100 12.1M    0     0  7205k      0  0:00:01  0:00:01 --:--:-- 18.2M
/root/.lein/self-installs/leiningen-2.10.0-standalone.jar.pending: OK
Leiningen 2.10.0 on Java 11.0.18 OpenJDK 64-Bit Server VM


In [16]:
!pip install streamparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.4/643.4 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.4/519

## Initial Configuration and Execution

Here you can see Streamparse in action, using a sample word count topology.

In [17]:
import os

os.environ['LEIN_ROOT']='1'
os.environ['PATH'] = os.environ['PATH'] + ':/content/apache-storm-1.2.4/bin/:/content'

In [18]:
!sparse quickstart wordcount


Creating your wordcount streamparse project...
    create    wordcount
    create    wordcount/.gitignore
    create    wordcount/config.json
    create    wordcount/fabfile.py
    create    wordcount/project.clj
    create    wordcount/README.md
    create    wordcount/src
    create    wordcount/src/bolts
    create    wordcount/src/bolts/__init__.py
    create    wordcount/src/bolts/wordcount.py
    create    wordcount/src/spouts
    create    wordcount/src/spouts/__init__.py
    create    wordcount/src/spouts/words.py
    create    wordcount/topologies
    create    wordcount/topologies/wordcount.py
    create    wordcount/virtualenvs
    create    wordcount/virtualenvs/wordcount.txt
Done.

Try running your topology locally with:

	cd wordcount
	sparse run


We need to make a quick fix to tell StreamParse to use Storm 1.2.3 instead of 1.1.0...

In [19]:
!cat wordcount/project.clj | sed s/1.1.0/1.2.4/ > wordcount/project2.clj

In [20]:
!mv wordcount/project2.clj wordcount/project.clj

## Running StreamParse for Real

Run this code and see the output...  At some point you'll want to interrupt it.  Press [Ctrl]-[M][I] when you are done.

In [21]:
!cd /content/wordcount ; sparse run



Traceback (most recent call last):
  File "/usr/local/bin/sparse", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.9/dist-packages/streamparse/cli/sparse.py", line 85, in main
    args.func(args)
  File "/usr/local/lib/python3.9/dist-packages/streamparse/cli/run.py", line 125, in main
    run_local_topology(
  File "/usr/local/lib/python3.9/dist-packages/streamparse/cli/run.py", line 61, in run_local_topology
    local_version = local_storm_version()
  File "/usr/local/lib/python3.9/dist-packages/streamparse/util.py", line 373, in local_storm_version
    raise RuntimeError(
RuntimeError: Unable to run 'storm version'!
STDOUT:
Need python version > 2.6
STDERR:



# Streaming, Joins, and Learning

Let's take a look at how we might use StreamParse spouts and bolts to implement a stream-based incremental machine learning application, for our airline flight data from the High-Level Streaming application.

Here we'll use a combination of Pandas (and PandaSQL) for our incremental joins and grouping, and Creme for incremental machine learning!

In [22]:
!pip install pandasql
!pip install creme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=218268e897d6bc0f2be5079a8b69b00379d4b8b302a4eac202978a4861265f21
  Stored in directory: /root/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.7/524.7 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for creme: filename=creme-0.6.1-cp39-cp39-linux_x86_64.whl size=1146063 sha256=5c3d15c370e6d49417463a691ec0eb4f9b1a1d926392c4d72933508f8a468fe9
  Stored in directory: /root/.cache/pip/wheels/a5/65/47/db774bf8e776dbe6896f94184365934a1656debdb4b524d700
  Created wheel for mmh3: filename=mmh3-2.

Let's get our data...

In [23]:
!pip install googledrivedownloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from google_drive_downloader import GoogleDriveDownloader as gdd

# One month's flights
gdd.download_file_from_google_drive(file_id='1PPtjGx8lr_cDUfVa3qwlk1W8yY6hY91n',
                                    dest_path='/content/ontime.csv')

# Airports
gdd.download_file_from_google_drive(file_id='1Qe4FpLg473FjfVhdRGGO4dHLmJa1ETq4',
                                    dest_path='/content/airports.csv')


## Our Spout

Let's define a spout.  To actually run this in StreamParse we would need to put it in a separate source file, which is inconvenient on Colab.

So instead we'll simulate StreamParse and watch what happens.

This spout reads one row at a time from the `ontime` dataframe.

In [25]:
from streamparse.spout import Spout
import pandas as pd

stream = []

#class FlightSpout:(Spout):
class FlightSpout:
  outputs=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED']
  # Position of next tuple in dataframe
  inx = 0
  # Overall stream tuple id
  tid = 0

  # Open the file as a dataframe
  def initialize(self, stormconf, context):
    self.ontime_df = pd.read_csv('/content/ontime.csv')
    self.ontime_df.dropna(inplace=True,subset=['ARR_DELAY_NEW'])

  # Read one row, increment the pointer, return the
  # row as a list
  def next_tuple(self):
    tup = self.ontime_df.iloc[self.inx]
    self.inx = self.inx + 1

    # We'll wrap around if we exceed the size of the table
    if self.inx >= len(self.ontime_df):
      self.inx = 0

    # We have to turn the series into a list of columns.
    # There is also an extra, blank column in the file
    self.emit(tup.tolist()[0:-1], tup_id=self.tid)
    self.tid = self.tid + 1

  # If a tuple was processed properly, do nothing
  def ack(self, tup_id):
    pass

  # If a tuple was processed incorrectly, we'll still
  # do nothing
  def fail(self, tup_id):
    pass

  # This would be omitted in the real code
  def emit(self, tuple, tup_id):
    global stream
    stream = stream + [tuple]

## A Bolt that Joins

As we get tuples, we'll want to join the flight with latitude and longitude information for origin and destination.  We can do this by preloading the `airports` as a dataframe, and incrementally joining tuples as they arrive.

In [26]:
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
from pandasql import sqldf

stream2 = []

#class JoinBolt(Bolt):

class JoinBolt:
  outputs=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','ARR_DELAY']

  def initialize(self, stormconf, context):
    self.airports_df = pd.read_csv('/content/airports.csv')

  def process(self, tup):
    flights_df = pd.DataFrame([tup],columns=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED'])
    airports_df = self.airports_df
    incremental_results = \
      sqldf('select YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM, ORIGIN, DEST, org.LATITUDE AS FROM_LAT, '\
            'org.LONGITUDE AS FROM_LONG, '\
            'dst.LATITUDE AS TO_LAT, dst.LONGITUDE AS TO_LONG, ARR_DELAY '\
            'from flights_df f join airports_df org on f.origin=org.IATA_CODE '\
            'join airports_df dst on f.dest=dst.IATA_CODE'
            ,locals())
    self.emit(incremental_results.iloc[0].tolist())

  def emit(self, tuple):
    global stream2
    stream2 = stream2 + [tuple]


## Windowed Grouping

Let's group by day.  We are going to assume that the stream is well-behaved in that days are monotonically increasing in the stream.

Thus, we buffer up tuples as long as they have the same day.  Once the day switches, we group them by the day and compute our statistics, then emit that.

In [27]:
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
from pandasql import sqldf

stream3 = []

class GroupBolt:#(Bolt):
  outputs = ['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST',\
                             'FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','AVG_DELAY']

  def initialize(self, stormconf, context):
    self.groups = pd.DataFrame([],columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','ARR_DELAY'])
    self.last_year = None
    self.last_month = None
    self.last_day_of_month = None

  def process(self, tup):
    flights_df = pd.DataFrame([tup],columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','ARR_DELAY'])

    if flights_df.iloc[0,0] != self.last_year or flights_df.iloc[0,1] != self.last_month \
      or flights_df.iloc[0,2] != self.last_day_of_month:
      print('** DONE WITH {}-{}-{} with {} entries'.format(self.last_year,self.last_month,self.last_day_of_month,len(self.groups)))
      if len(self.groups) > 0:
        groups_df = self.groups
        grouped_results = sqldf('select YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST, '\
                        'FROM_LAT,FROM_LONG,TO_LAT,TO_LONG, COUNT(ARR_DELAY) as NbrFlights, avg(ARR_DELAY) as avg_delay '\
                        'FROM groups_df GROUP BY YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST,FROM_LAT,FROM_LONG,TO_LAT,TO_LONG '\
                        'ORDER BY CARRIER,FL_NUM,ORIGIN,DEST',
                        locals())
        
        for result in grouped_results.itertuples(index=False):
          self.emit(list(result))

      self.groups = flights_df
      self.last_year = flights_df.iloc[0,0]
      self.last_month = flights_df.iloc[0,1]
      self.last_day_of_month = flights_df.iloc[0,2]
    else:
      self.groups = pd.concat([self.groups, flights_df])

  def emit(self, tuple):
    global stream3
    stream3 = stream3 + [tuple]


## Incremental Learning via Creme

We want to incrementally train a linear regression algorithm to predict our delays.

The Creme package (https://creme-ml.github.io/) allows us to do incrementally process data, using stochastic gradient descent-style methods to incrementally perform linear regression.  It also has incremental methods for one-hot encoding, scaling, etc.

In [28]:
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
import creme.preprocessing
import creme.linear_model

stream4 = []

class LearnBolt:#(Bolt):
  outputs = ['predicted','actual','error']

  def initialize(self, stormconf, context):
    self.scaler = creme.preprocessing.StandardScaler()
    self.lin_reg = creme.linear_model.LinearRegression()
    self.carrier_one_hot = creme.preprocessing.OneHotEncoder('CARRIER', sparse=True)
    self.origin_one_hot = creme.preprocessing.OneHotEncoder('ORIGIN', sparse=True)
    self.dest_one_hot = creme.preprocessing.OneHotEncoder('DEST', sparse=True)

  def process(self, tup):
    x = pd.Series(tup, index=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST',\
                             'FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','AVG_DELAY'])
    x_onehot = pd.Series(self.carrier_one_hot.fit_one(x).transform_one(x))
    y = x['AVG_DELAY']
    x = x.append(x_onehot).drop(['CARRIER','AVG_DELAY'])
    x_onehot = pd.Series(self.origin_one_hot.fit_one(x).transform_one(x))
    x = x.append(x_onehot).drop(['ORIGIN'])
    x_onehot = pd.Series(self.dest_one_hot.fit_one(x).transform_one(x))
    x = x.append(x_onehot).drop(['DEST'])

    x = self.scaler.fit_one(x).transform_one(x)
    yhat = self.lin_reg.predict_one(x)
    print('Predicted {}, actual {}, error {}'.format(yhat, y, abs(yhat - y)))
    self.emit([yhat,y,abs(yhat-y)])

    self.lin_reg.fit_one(x, y)


  def emit(self, tuple):
    global stream4
    stream4 = stream4 + [tuple]


In [29]:
stream = []
fs = FlightSpout()

fs.initialize(None, None)
for i in range(2000):
  fs.next_tuple()

pd.DataFrame(stream,columns=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED'])

,YEAR,MONTH,DAY_OF_MONTH,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN,DEST,ARR_DELAY,CANCELLED
0,2018,1,2,19393,WN,1325,SJU,MCO,0.0,0.0
1,2018,1,2,19393,WN,5159,SJU,MCO,0.0,0.0
2,2018,1,2,19393,WN,5890,SJU,MCO,9.0,0.0
3,2018,1,2,19393,WN,6618,SJU,MCO,0.0,0.0
4,2018,1,2,19393,WN,1701,SJU,MDW,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1995,2018,1,3,19393,WN,762,LAS,OAK,55.0,0.0
1996,2018,1,3,19393,WN,1029,LAS,OAK,0.0,0.0
1997,2018,1,3,19393,WN,1358,LAS,OAK,18.0,0.0
1998,2018,1,3,19393,WN,1396,LAS,OAK,0.0,0.0


In [32]:
stream2 = []
jb = JoinBolt()

jb.initialize(None, None)

for e in stream:
  jb.process(e)

pd.DataFrame(stream2,columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','ARR_DELAY'])

ObjectNotExecutableError: ignored

In [31]:
stream3 = []
gb = GroupBolt()

gb.initialize(None, None)

for e in stream2:
  gb.process(e)

pd.DataFrame(stream3,columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST',\
                             'FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','AVG_DELAY'])

,YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST,FROM_LAT,FROM_LONG,TO_LAT,TO_LONG,NBR_FLIGHTS,AVG_DELAY


In [ ]:
stream4 = []
lb = LearnBolt()

lb.initialize(None, None)

for e in stream3:
  lb.process(e)

pd.DataFrame(stream4,columns=['predicted','actual','error'])

Predicted 0.0, actual 0.0, error 0.0
Predicted 0.0, actual 0.0, error 0.0
Predicted 0.0, actual 5.0, error 5.0
Predicted 0.11506785759732426, actual 19.0, error 18.884932142402675
Predicted 1.0301725324668687, actual 6.0, error 4.969827467533131
Predicted 1.0147117000442663, actual 0.0, error 1.0147117000442663
Predicted 1.6955509700847504, actual 0.0, error 1.6955509700847504
Predicted 1.3583920872662598, actual 0.0, error 1.3583920872662598
Predicted 1.1291731455517737, actual 0.0, error 1.1291731455517737
Predicted 1.0565963343981761, actual 0.0, error 1.0565963343981761
Predicted 0.9438499206259802, actual 6.0, error 5.05615007937402
Predicted 1.650476895487974, actual 0.0, error 1.650476895487974
Predicted 1.4574779284218136, actual 6.0, error 4.542522071578187
Predicted 1.7858917214543037, actual 0.0, error 1.7858917214543037
Predicted 1.2666363875668571, actual 0.0, error 1.2666363875668571
Predicted 1.017426845539113, actual 1.0, error 0.01742684553911289
Predicted 0.9666246618

,predicted,actual,error
0,0.000000,0.0,0.000000
1,0.000000,0.0,0.000000
2,0.000000,5.0,5.000000
3,0.115068,19.0,18.884932
4,1.030173,6.0,4.969827
...,...,...,...
378,9.584332,25.0,15.415668
379,9.729228,8.0,1.729228
380,10.207437,10.0,0.207437
381,9.680903,0.0,9.680903
